In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from emu.pipeline.remote import RemoteCSV
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

from tqdm import tqdm_notebook as tqdm

In [2]:
all_files = get_data_manifest()

In [3]:
seeg_root = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/')

nev_path = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.Events_0007.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'PO_Day_02.*_0007.ncs')))

# ncs = nlx.load_ncs()
print('nev path: ',nev_path)
print('ncs_paths:')
for p in ncs_paths[:10]:
    print(p)

nev path:  /home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.Events_0007.nev
ncs_paths:
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC100_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC101_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC102_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC103_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC104_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC105_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC106_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC107_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC108_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC109_0007.ncs


In [4]:
p = Participant(patient_id=1,raw_files=all_files,seeg_raw_path=seeg_root)

/usr/local/lib/python3.6/dist-packages/luigi/parameter.py:283: UserWarning: Parameter "file_path" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))


In [5]:
trodes = p.electrode_locations.load(force=True)

In [6]:
from pynwb.ecephys import ElectricalSeries, ElectrodeGroup,Device
from pynwb.file import ElectrodeTable
from hdmf.common import DynamicTableRegion

In [7]:
def gen_electrode_groups(electrodes_df, group_col='wire_num'):
    
    wire_loc = electrodes_df[[group_col,'anat_lg']].drop_duplicates()
    
    electrode_groups = [ElectrodeGroup('wire_{}'.format(wire.wire_num),'Wire',wire.anat_lg,dev1) for i,wire in wire_loc.iterrows()]
    
    for grouped, e_grp in zip(electrodes_df.groupby(group_col),electrode_groups):
        name,group = grouped
        for i,row in group.iterrows():
            table.add_row(id=row.chan_num,x=0,y=0,z=0,imp=-1,location=row.anat_sh,filtering='none',
                         group=e_grp, group_name='wire_{}'.format(row.wire_num))
        
        yield e_grp

In [8]:
table = ElectrodeTable('electrodes','Epilepsy Monitoring Unit Electrodes')
dev1 = Device('Neuralynx')
egrp_generator = gen_electrode_groups(trodes,)

In [9]:
e_groups = list(gen_electrode_groups(trodes))

In [10]:
table.to_dataframe().head()

,x,y,z,imp,location,filtering,group,group_name
id,,,,,,,,
1,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
2,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
3,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
4,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
5,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1


In [11]:
# region = DynamicTableRegion('electrodes', [0, 2], 'the first and third electrodes', table)

region = DynamicTableRegion('electrodes', list(np.arange(15)), 'wire_1 electrodes', table)

In [12]:
region

In [13]:
get_channel = lambda ch,filepaths: [p for p in filepaths if 'CSC{}_'.format(str(ch)) in p]
g_channels = []
for i in trodes.iloc[:16].chan_num.values:
    g_channels.extend(get_channel(i,ncs_paths))
data = [nlx.load_ncs(p)['data'].astype(np.float16) for p in g_channels]


/home/elijahc/dev/emu/emu/neuralynx_io.py:79: UserWarning: Unable to parse original file path from Neuralynx header: -FileType NCS
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.4
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:131: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID 81411b1d-4887-44e9-9f66-c60b61009826
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line from Neuralynx header: -ProbeName
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/dev/emu/emu/neuralynx_io.py:93: UserWarning: Unable to parse parameter line from Neuralynx header: -T

(15, 54716416)

In [15]:
data = np.swapaxes(np.stack(data),0,1)

In [18]:
eS = ElectricalSeries('wire_1', data, region, rate=1000.0)

In [42]:
table.to_dataframe()

,x,y,z,imp,location,filtering,group,group_name
id,,,,,,,,
1,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
2,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
3,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
4,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
5,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
6,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
7,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
8,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1
9,0,0,0,-1,LOF,none,wire_1 pynwb.ecephys.ElectrodeGroup at 0x13978...,wire_1


In [43]:
len(ncs_paths)

177

In [45]:
sorted(ncs_paths)

['/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC100_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC101_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC102_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC103_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC104_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC105_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC106_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC107_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC108_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC109_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC10_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC110_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC111_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC112_0007.ncs',
 '/home/elijahc/.emu/pdil/pt_01/SEE